**Import libraries**

In [1]:
import holoviews as hv
from holoviews import opts
from holoviews import streams
import xarray as xr
import hvplot.pandas
import hvplot.xarray
import geopandas as gpd
import panel as pn
import pandas as pd
import numpy as np
from cmcrameri import cm
import cmocean

pn.extension('tabulator', sizing_mode="stretch_width")
hv.extension('bokeh')

**Figure 1**

In [2]:
# Shapefile
fig1_philippines = (
    gpd.read_file('shapefile/climatezones/climate_type.shp')
    .replace({'Type': {4.0: 'Type IV', 3.0: 'Type III', 2.0: 'Type II', 1.0: 'Type I'}})
    .iloc[::-1]
)

# Data
fig1_file_paths = ['df_precip.csv', 'df_tas.csv', 'df_tasmin.csv', 'df_tasmax.csv']
fig1_dfs = {name: pd.read_csv(f"data/{file}").drop(columns=['Unnamed: 0']).melt(
        id_vars=['Month'], var_name='Type', value_name='Value'
    )
    for name, file in zip(['precip', 'tas', 'tasmin', 'tasmax'], fig1_file_paths)
}
fig1_df_precip, fig1_df_tas, fig1_df_tasmin, fig1_df_tasmax = fig1_dfs.values()

# Colour Palette
fig1_cmap = {
    'Type I': 'red',
    'Type II': 'blue',
    'Type III': 'gold',
    'Type IV': 'limegreen'
}

# Map Plot with Stream for Click Detection
fig1_map_plot = fig1_philippines.hvplot(
    geo=True, color='Type', cmap=fig1_cmap, hover_cols=['Type'], width=500, height=900, legend='top_right'
).opts(
    opts.Polygons(line_color='black', tools=['hover', 'tap'], active_tools=['wheel_zoom'], toolbar='above')
)

# Add Selection Stream
selection = streams.Selection1D(source=fig1_map_plot)

# Function to update plots dynamically
def update_plots(indexes):
    if not indexes:
        selected_types = ['Type I', 'Type II', 'Type III', 'Type IV']  # Default to all
    else:
        selected_types = fig1_philippines.iloc[indexes]['Type'].tolist()

    # --- Precipitation Plot ---
    precip_plots = []
    for climate_type in selected_types:
        precip_plot = fig1_df_precip[fig1_df_precip['Type'] == climate_type].hvplot.bar(
            x='Month', y='Value', label=climate_type, color=fig1_cmap[climate_type],
            width=400, height=450, alpha=1.0
        ).opts(
            opts.Bars(
                tools=['hover', 'tap'], active_tools=['wheel_zoom'], toolbar='above',
                title='Monthly Precipitation',
                ylabel='Mean Precipitation (mm)'
            )
        )
        precip_plots.append(precip_plot)

    precip_combined = hv.Overlay(precip_plots).opts(legend_position='top', axiswise=True)

    # --- Temperature Plot ---
    temp_plots = []
    for climate_type in selected_types:
        temp_plot = fig1_df_tas[fig1_df_tas['Type'] == climate_type].hvplot.line(
            x='Month', y='Value', label=climate_type, color=fig1_cmap[climate_type],
            width=400, height=450
        ).opts(
            opts.Curve(
                tools=['hover', 'tap'], active_tools=['wheel_zoom'], toolbar='above',
                title='Monthly Temperature',
                ylabel='Temperature (°C)'
            )
        )
        tasmin_plot = fig1_df_tasmin[fig1_df_tasmin['Type'] == climate_type].hvplot.line(
            x='Month', y='Value', label=climate_type, color=fig1_cmap[climate_type],
            width=400, height=450, line_dash='dashed'
        )
        tasmax_plot = fig1_df_tasmax[fig1_df_tasmax['Type'] == climate_type].hvplot.line(
            x='Month', y='Value', label=climate_type, color=fig1_cmap[climate_type],
            width=400, height=450, line_dash='dotted'
        )
        temp_plot = temp_plot * tasmin_plot * tasmax_plot
        temp_plots.append(temp_plot)

    temp_combined = hv.Overlay(temp_plots).opts(legend_position='top', axiswise=True)

    return pn.Column(precip_combined, temp_combined)

# Bind the update function to selection
fig1_dynamic_plot = pn.bind(update_plots, selection.param.index)

# Create the Final Layout
fig1_layout = pn.Column(
    pn.Row(
        fig1_map_plot,
        fig1_dynamic_plot
    ),
    width=1280,
)

**Figure 2**

In [3]:
# Create data for number of landfalling TCs and number of non-landfalling TCs in the Philippines from 1951 to 2013
fig2_data = {
    'Month': ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
    'Non-landfalling': [14, 13, 9, 11, 34, 50, 126, 138, 111, 71, 42, 37],
    'Landfalling': [12, 7, 10, 18, 31, 50, 80, 65, 78, 88, 81, 44]
}
fig2_df = pd.DataFrame(fig2_data)

# Create a stacked bar chart
fig2_plot = fig2_df.hvplot.bar(
    x='Month',
    y=['Non-landfalling', 'Landfalling'],  # Specify both columns for stacking
    stacked=True,  # Enable stacking
    color=['cornflowerblue', 'royalblue'],  # Colors for each category
    alpha=0.8,
    width=1280,
    height=720,
    legend='top_left',
).opts(
    xlabel='Month',
    ylabel='Number of TCs',
    title='Number of Landfalling and Non-Landfalling TCs \n in the Philippines (1951-2013)',
    tools=['hover', 'tap'],
    toolbar='right'
)

# Add text labels for each bar
labels_data = []
for i, row in fig2_df.iterrows():
    # Add labels for Non-landfalling
    labels_data.append({'Month': row['Month'], 'y': row['Non-landfalling'] / 2, 'label': str(row['Non-landfalling'])})
    # Add labels for Landfalling
    labels_data.append({'Month': row['Month'], 'y': row['Non-landfalling'] + row['Landfalling'] / 2, 'label': str(row['Landfalling'])})

labels_df = pd.DataFrame(labels_data)

# Create labels
labels = hv.Labels(labels_df, ['Month', 'y'], 'label').opts(
    text_align='center',
    text_baseline='middle',
    text_color='black',
    text_font_size='8pt'
)

# Overlay the bar chart and the labels
fig2_plot_with_labels = fig2_plot * labels

**Figure 4**

In [4]:
fig4_cebu_shapefile = gpd.read_file('shapefile/phl_adm_psa_namria_20231106_shp/phl_admbnda_adm2_psa_namria_20231106.shp')
fig4_cebu = fig4_cebu_shapefile[fig4_cebu_shapefile.ADM2_EN == 'Cebu']
fig4_ncr_shapefile = gpd.read_file('shapefile/phl_adm_psa_namria_20231106_shp/phl_admbnda_adm1_psa_namria_20231106.shp')
fig4_ncr = fig4_ncr_shapefile[fig4_ncr_shapefile.ADM1_EN == 'National Capital Region (NCR)']
fig4_cebu_precip_df = pd.read_csv('data/fig4_cebu_precip.csv')
fig4_cebu_tas_df = pd.read_csv('data/fig4_cebu_tas.csv')
fig4_ncr_precip_df = pd.read_csv('data/fig4_manila_precip.csv')
fig4_ncr_tas_df = pd.read_csv('data/fig4_manila_tas.csv')

fig4_cebu_map = fig4_cebu.hvplot(
    geo=True, color='gold', alpha=1.0, line_color='black', width=400, height=400
    ).opts(title='Cebu', toolbar='left', active_tools=['wheel_zoom'], tools=['hover', 'tap'])
fig4_ncr_map = fig4_ncr.hvplot(
    geo=True, color='red', alpha=1.0, line_color='black', width=400, height=400
    ).opts(title='NCR (Metropolitan Manila)', toolbar='left', active_tools=['wheel_zoom'], tools=['hover', 'tap'])

# Process Data
el_nino = {2003, 2007, 2010, 2016}
la_nina = {2008, 2011, 2012, 2021, 2022}
def process_precip_data(df):
    df_elnino = df[df['Year'].isin(el_nino)].copy()
    df_lanina = df[df['Year'].isin(la_nina)].copy()
    df_neutral = df[~df['Year'].isin(el_nino.union(la_nina))].copy()
    
    # Use .loc to avoid SettingWithCopyWarning
    for d in [df_elnino, df_lanina, df_neutral]:
        d.loc[:, 'Annual Precipitation (mm)'] = d['Annual Precipitation (mm)'].round(0).astype(int)
    
    return df_elnino, df_lanina, df_neutral

cebu_precip_elnino, cebu_precip_lanina, cebu_precip_neutral = process_precip_data(fig4_cebu_precip_df)
ncr_precip_elnino, ncr_precip_lanina, ncr_precip_neutral = process_precip_data(fig4_ncr_precip_df)

def fig4_plot_precipitation(df_neutral, df_elnino, df_lanina, region_name):
    # Plot neutral 
    neutral_plot = df_neutral.hvplot.bar(x='Year', y='Annual Precipitation (mm)',
                                        color='silver', label='Neutral',
                                        alpha=0.8, bar_width=4,
                                        width=400, height=450)
    # Plot El Niño
    elnino_plot = df_elnino.hvplot.bar(x='Year', y='Annual Precipitation (mm)',
                                    color='orange', label='El Niño',
                                    alpha=0.8, bar_width=4.5,
                                    width=400, height=450)
    # Plot La Niña
    lanina_plot = df_lanina.hvplot.bar(x='Year', y='Annual Precipitation (mm)',
                                    color='cadetblue', label='La Niña',
                                    alpha=0.8, bar_width=3,
                                    width=400, height=450)
    # Overlay the plots
    combined_plot = (neutral_plot * elnino_plot * lanina_plot).opts(
        title=f'Observed Annual Precipitation in {region_name}',
        xlabel='Year',
        ylabel='Precipitation (mm)',
        legend_position='top',
        tools=['hover', 'tap'],
        active_tools=['wheel_zoom'],
        toolbar='right',
        show_grid=False,
    )

    return combined_plot

fig4_cebu_precip_plot = fig4_plot_precipitation(cebu_precip_neutral, cebu_precip_elnino, cebu_precip_lanina, 'Cebu')
fig4_ncr_precip_plot = fig4_plot_precipitation(ncr_precip_neutral, ncr_precip_elnino, ncr_precip_lanina, 'NCR')

# Temperature plot
def fig4_plot_temperature(df_tas, region_name):
    # Plot temperature
    temp_plot = df_tas.hvplot.line(x='Year', y='Tas',
                                color='red',
                                width=400, height=450).opts(
                                    title=f'Observed Annual Temperature in {region_name}',
                                    xlabel='Year',
                                    ylabel='Temperature (°C)',
                                    tools=['hover', 'tap'],
                                    active_tools=['wheel_zoom'],
                                    toolbar='right',
                                    show_grid=False,
                                )
    # Add points for each year
    points = df_tas.hvplot.scatter(x='Year', y='Tas', color='black', size=15)

    # Overlay the plots
    combined_plot = temp_plot * points 
    return combined_plot

fig4_cebu_temp_plot = fig4_plot_temperature(fig4_cebu_tas_df,'Cebu')
fig4_ncr_temp_plot = fig4_plot_temperature(fig4_ncr_tas_df,'NCR')

# Figure 4 Layout
fig4_layout_cebu = pn.Row(fig4_cebu_map, fig4_cebu_precip_plot, fig4_cebu_temp_plot)
fig4_layout_ncr = pn.Row(fig4_ncr_map, fig4_ncr_precip_plot, fig4_ncr_temp_plot)

**Figure 5**

In [5]:
fig5_manila_shpaefile = gpd.read_file('shapefile/phl_adm_psa_namria_20231106_shp/phl_admbnda_adm3_psa_namria_20231106.shp')
fig5_manila = fig5_manila_shpaefile[fig5_manila_shpaefile.ADM3_EN == 'City of Manila']
fig5_cebu_city = fig5_manila_shpaefile[fig5_manila_shpaefile.ADM3_EN == 'Cebu City (Capital)']
fig5_cebu_rx1day_df = pd.read_csv('data/cebu_rx1day.csv')
fig5_manila_rx1day_df = pd.read_csv('data/manila_rx1day.csv')
fig5_cebu_city_rx1day_df = pd.read_csv('data/cebu_city_rx1day.csv')
fig5_ncr_rx1day_df = pd.read_csv('data/ncr_rx1day.csv')
fig5_cebu_txx_df = pd.read_csv('data/cebu_TXx.csv')
fig5_cebu_city_txx_df = pd.read_csv('data/cebu_city_TXx.csv')
fig5_manila_txx_df = pd.read_csv('data/manila_TXx.csv')
fig5_ncr_txx_df = pd.read_csv('data/ncr_TXx.csv')

# Drop the 'Unnamed: 0' column
def drop_unnamed(df):
    return df.drop(columns=['Unnamed: 0'], errors='ignore')
dfs = [
    "fig5_cebu_city_rx1day_df", "fig5_ncr_rx1day_df", "fig5_manila_rx1day_df", "fig5_cebu_rx1day_df",
    "fig5_cebu_txx_df", "fig5_cebu_city_txx_df", "fig5_manila_txx_df", "fig5_ncr_txx_df"
]
for df_name in dfs:
    globals()[df_name] = drop_unnamed(globals()[df_name])

# Process Data
def fig5_process_precip_data(df):
    df_elnino = df[df['Year'].isin(el_nino)].copy()
    df_lanina = df[df['Year'].isin(la_nina)].copy()
    df_neutral = df[~df['Year'].isin(el_nino.union(la_nina))].copy()
    
    # Use .loc to avoid SettingWithCopyWarning
    for d in [df_elnino, df_lanina, df_neutral]:
        d.loc[:, 'rx1day'] = d['rx1day'].round(0).astype(int)
    
    return df_elnino, df_lanina, df_neutral

fig5_cebu_rx1day_el_nino, fig5_cebu_rx1day_la_nina, fig5_cebu_rx1day_neutral = fig5_process_precip_data(fig5_cebu_rx1day_df)
fig5_cebu_city_rx1day_el_nino, fig5_cebu_city_rx1day_la_nina, fig5_cebu_city_rx1day_neutral = fig5_process_precip_data(fig5_cebu_city_rx1day_df)
fig5_manila_rx1day_el_nino, fig5_manila_rx1day_la_nina, fig5_manila_rx1day_neutral = fig5_process_precip_data(fig5_manila_rx1day_df)
fig5_ncr_rx1day_el_nino, fig5_ncr_rx1day_la_nina, fig5_ncr_rx1day_neutral = fig5_process_precip_data(fig5_ncr_rx1day_df)

fig5_cebu_txx_df['TXx'] = fig5_cebu_txx_df['TXx'].round(1)
fig5_manila_txx_df['TXx'] = fig5_manila_txx_df['TXx'].round(1)
fig5_cebu_city_txx_df['TXx'] = fig5_cebu_city_txx_df['TXx'].round(1)
fig5_ncr_txx_df['TXx'] = fig5_ncr_txx_df['TXx'].round(1)

# Plot the map
fig5_manila_map = fig5_manila.hvplot(
    geo=True, color='red', alpha=1.0, line_color='black',
    width=400, height=400, hover_cols=['Manila']
)
fig5_cebu_city_map = fig5_cebu_city.hvplot(
    geo=True, color='red', alpha=1.0, line_color='black',
    width=400, height=400, hover_cols=['Cebu City']
)
fig5_ncr_map = fig4_ncr.hvplot(
    geo=True, color='lightgrey', alpha=0.5, line_color='black',
    width=400, height=400, hover_cols=['National Capital Region (NCR)']
)
fig5_cebu_map = fig4_cebu.hvplot(
    geo=True, color='lightgrey', alpha=0.5, line_color='black',
    width=400, height=400, hover_cols=['Cebu']
)

fig5_map = fig5_manila_map * fig5_cebu_city_map * fig5_ncr_map * fig5_cebu_map

fig5_map.opts(
    title='Areas of Interest',
    toolbar='left',
    active_tools=['wheel_zoom'],
    tools=['hover', 'tap']
)

# Precipitation plot
def fig5_plot_precipitation(df_neutral, df_elnino, df_lanina, region_name):
    # Plot neutral 
    neutral_plot = df_neutral.hvplot.bar(x='Year', y='rx1day',
                                        color='silver', label='Neutral',
                                        alpha=0.8, bar_width=4,
                                        width=400, height=450)
    # Plot El Niño
    elnino_plot = df_elnino.hvplot.bar(x='Year', y='rx1day',
                                    color='orange', label='El Niño',
                                    alpha=0.8, bar_width=4.5,
                                    width=400, height=450)
    # Plot La Niña
    lanina_plot = df_lanina.hvplot.bar(x='Year', y='rx1day',
                                    color='cadetblue', label='La Niña',
                                    alpha=0.8, bar_width=3,
                                    width=400, height=450)
    # Overlay the plots
    combined_plot = (neutral_plot * elnino_plot * lanina_plot).opts(
        title=f'Observed Rx1day in {region_name}',
        xlabel='Year',
        ylabel='Precipitation (mm)',
        legend_position='top',
        tools=['hover', 'tap'],
        active_tools=['wheel_zoom'],
        toolbar='right',
        show_grid=False,
    )

    return combined_plot

fig5_cebu_rx1day_plot = fig5_plot_precipitation(fig5_cebu_rx1day_neutral, fig5_cebu_rx1day_el_nino, fig5_cebu_rx1day_la_nina, 'Cebu')
fig5_cebu_city_rx1day_plot = fig5_plot_precipitation(fig5_cebu_city_rx1day_neutral, fig5_cebu_city_rx1day_el_nino, fig5_cebu_city_rx1day_la_nina, 'Cebu City')
fig5_manila_rx1day_plot = fig5_plot_precipitation(fig5_manila_rx1day_neutral, fig5_manila_rx1day_el_nino, fig5_manila_rx1day_la_nina, 'Manila')
fig5_ncr_rx1day_plot = fig5_plot_precipitation(fig5_ncr_rx1day_neutral, fig5_ncr_rx1day_el_nino, fig5_ncr_rx1day_la_nina, 'NCR')

# Temperature plot
def fig5_plot_temp(df_txx, region_name):
    # Plot temperature
    temp_plot = df_txx.hvplot.line(x='Year', y='TXx',
                                color='red', 
                                width=400, height=450).opts(
                                    title=f'Observed TXx in {region_name}',
                                    xlabel='Year',
                                    ylabel='Temperature (°C)',
                                    tools=['hover', 'tap'],
                                    active_tools=['wheel_zoom'],
                                    toolbar='right',
                                    show_grid=False,
                                )
    # Add points for each year
    points = df_txx.hvplot.scatter(x='Year', y='TXx', color='black', size=15)

    # Overlay the plots
    combined_plot = temp_plot * points
    return combined_plot

fig5_cebu_txx_plot = fig5_plot_temp(fig5_cebu_txx_df, 'Cebu')
fig5_cebu_city_txx_plot = fig5_plot_temp(fig5_cebu_city_txx_df, 'Cebu City')
fig5_manila_txx_plot = fig5_plot_temp(fig5_manila_txx_df, 'Manila')
fig5_ncr_txx_plot = fig5_plot_temp(fig5_ncr_txx_df, 'NCR')

# Layout
fig5_layout_cebu = pn.Row(fig5_map, fig5_cebu_rx1day_plot, fig5_cebu_txx_plot)
fig5_layout_cebu_city = pn.Row(fig5_map, fig5_cebu_city_rx1day_plot, fig5_cebu_city_txx_plot)
fig5_layout_manila = pn.Row(fig5_map, fig5_manila_rx1day_plot, fig5_manila_txx_plot)
fig5_layout_ncr = pn.Row(fig5_map, fig5_ncr_rx1day_plot, fig5_ncr_txx_plot)

**Figure 8**

In [6]:
# Read historical and ssp585 data 
fig8_philippines_historical = xr.open_dataset('data/fig8_philippines_historical.nc')
fig8_philippines_ssp585_mid = xr.open_dataset('data/fig8_philippines_mid_ssp585.nc')
fig8_philippines_ssp585_end = xr.open_dataset('data/fig8_philippines_end_ssp585.nc')
fig8_manila_historical = xr.open_dataset('data/fig8_manila_historical.nc')
fig8_manila_ssp585_mid = xr.open_dataset('data/fig8_manila_mid_ssp585.nc')
fig8_manila_ssp585_end = xr.open_dataset('data/fig8_manila_end_ssp585.nc')
fig8_ncr_historical = xr.open_dataset('data/fig8_ncr_historical.nc')
fig8_ncr_ssp585_mid = xr.open_dataset('data/fig8_ncr_mid_ssp585.nc')
fig8_ncr_ssp585_end = xr.open_dataset('data/fig8_ncr_end_ssp585.nc')
fig8_cebu_historical = xr.open_dataset('data/fig8_cebu_historical.nc')
fig8_cebu_ssp585_mid = xr.open_dataset('data/fig8_cebu_mid_ssp585.nc')
fig8_cebu_ssp585_end = xr.open_dataset('data/fig8_cebu_end_ssp585.nc')
fig8_cebu_city_historical = xr.open_dataset('data/fig8_cebu_city_historical.nc')
fig8_cebu_city_ssp585_mid = xr.open_dataset('data/fig8_cebu_city_mid_ssp585.nc')
fig8_cebu_city_ssp585_end = xr.open_dataset('data/fig8_cebu_city_end_ssp585.nc')
fig8_cdd_philippines_historical = xr.open_dataset('data/Fig8_Philippines_Historical_CDD.nc')
fig8_cdd_philippines_ssp585_mid = xr.open_dataset('data/Fig8_Philippines_mid_ssp585_CDD.nc')
fig8_cdd_philippines_ssp585_end = xr.open_dataset('data/Fig8_Philippines_end_ssp585_CDD.nc')
fig8_cdd_manila_historical = xr.open_dataset('data/Fig8_Manila_Historical_CDD.nc')
fig8_cdd_manila_ssp585_mid = xr.open_dataset('data/Fig8_Manila_mid_ssp585_CDD.nc')
fig8_cdd_manila_ssp585_end = xr.open_dataset('data/Fig8_Manila_end_ssp585_CDD.nc')
fig8_cdd_ncr_historical = xr.open_dataset('data/Fig8_NCR_Historical_CDD.nc')
fig8_cdd_ncr_ssp585_mid = xr.open_dataset('data/Fig8_NCR_mid_ssp585_CDD.nc')
fig8_cdd_ncr_ssp585_end = xr.open_dataset('data/Fig8_NCR_end_ssp585_CDD.nc')
fig8_cdd_cebu_historical = xr.open_dataset('data/Fig8_Cebu_Historical_CDD.nc')
fig8_cdd_cebu_ssp585_mid = xr.open_dataset('data/Fig8_Cebu_mid_ssp585_CDD.nc')
fig8_cdd_cebu_ssp585_end = xr.open_dataset('data/Fig8_Cebu_end_ssp585_CDD.nc')
fig8_cdd_cebu_city_historical = xr.open_dataset('data/Fig8_Cebu_City_Historical_CDD.nc')
fig8_cdd_cebu_city_ssp585_mid = xr.open_dataset('data/Fig8_Cebu_City_mid_ssp585_CDD.nc')
fig8_cdd_cebu_city_ssp585_end = xr.open_dataset('data/Fig8_Cebu_City_end_ssp585_CDD.nc')

fig8_philippines = gpd.GeoSeries(fig1_philippines['geometry'].union_all())
fig8_philippines = gpd.GeoDataFrame(geometry=fig8_philippines)

# Define colormaps
colormap_historical = cmocean.cm.rain
colormap_sequential = cm.broc_r
colormap_non_sequential = cm.navia_r
cdd_colormap_historical = cm.lajolla_r
cdd_colormap_sequential = cm.vik
cdd_colormap_period = cm.buda_r
def historical_plot(data, variable, boundary, colormap, clabel):
    boundary_plot = boundary.hvplot(
        geo=True, 
        color=None,
        alpha=1,
        line_color='black',
        line_width=0.2,
        width=400,
        height=400)
    historical_plot = data[variable].hvplot(
        x='lon',
        y='lat',
        rasterize=True,
        geo=True,
        clabel=clabel,
        cmap=colormap,
        width=400,
        height=400,
        frame_width=400,
        frame_height=400
    ).opts(
        title='Historical Period (1995-2014)',
        xlabel='Longitude',
        ylabel='Latitude',
        xticks=5,
        yticks=2,
        tools=['hover','wheel_zoom'],
        toolbar='right',
        colorbar_position='bottom',
    )
    return historical_plot * boundary_plot

# Historical plot
fig8_philippines_historical_plot = historical_plot(fig8_philippines_historical, 'RX1day', fig8_philippines, colormap_historical, 'RX1day (mm/day)')
fig8_manila_historical_plot = historical_plot(fig8_manila_historical, 'RX1day', fig5_manila, colormap_historical, 'RX1day (mm/day)')
fig8_cebu_historical_plot = historical_plot(fig8_cebu_historical, 'RX1day', fig4_cebu, colormap_historical, 'RX1day (mm/day)')
fig8_cebu_city_historical_plot = historical_plot(fig8_cebu_city_historical, 'RX1day', fig5_cebu_city, colormap_historical, 'RX1day (mm/day)')
fig8_ncr_historical_plot = historical_plot(fig8_ncr_historical, 'RX1day', fig4_ncr, colormap_historical, 'RX1day (mm/day)')

fig8_philippines_cdd_historical_plot = historical_plot(fig8_cdd_philippines_historical, 'CDD', fig8_philippines, cdd_colormap_historical, 'CDD (days)')
fig8_manila_cdd_historical_plot = historical_plot(fig8_cdd_manila_historical, 'CDD', fig5_manila, cdd_colormap_historical, 'CDD (days)')
fig8_cebu_cdd_historical_plot = historical_plot(fig8_cdd_cebu_historical, 'CDD', fig4_cebu, cdd_colormap_historical, 'CDD (days)')
fig8_cebu_city_cdd_historical_plot = historical_plot(fig8_cdd_cebu_city_historical, 'CDD', fig5_cebu_city, cdd_colormap_historical, 'CDD (days)')
fig8_ncr_cdd_historical_plot = historical_plot(fig8_cdd_ncr_historical, 'CDD', fig4_ncr, cdd_colormap_historical, 'CDD (days)')

fig8_philippines_cdd_period_historical_plot = historical_plot(fig8_cdd_philippines_historical,  'number_of_cdd_periods_with_more_than_5days_per_time_period', fig8_philippines, cdd_colormap_period, 'Number of CDD periods with more than 5 days per time period')
fig8_manila_cdd_period_historical_plot = historical_plot(fig8_cdd_manila_historical,  'number_of_cdd_periods_with_more_than_5days_per_time_period', fig5_manila, cdd_colormap_period, 'Number of CDD periods with more than 5 days per time period')
fig8_cebu_cdd_period_historical_plot = historical_plot(fig8_cdd_cebu_historical,  'number_of_cdd_periods_with_more_than_5days_per_time_period', fig4_cebu, cdd_colormap_period, 'Number of CDD periods with more than 5 days per time period')
fig8_cebu_city_cdd_period_historical_plot = historical_plot(fig8_cdd_cebu_city_historical,  'number_of_cdd_periods_with_more_than_5days_per_time_period', fig5_cebu_city, cdd_colormap_period, 'Number of CDD periods with more than 5 days per time period')
fig8_ncr_cdd_period_historical_plot = historical_plot(fig8_cdd_ncr_historical,  'number_of_cdd_periods_with_more_than_5days_per_time_period', fig4_ncr, cdd_colormap_period, 'Number of CDD periods with more than 5 days per time period')

# ssp585 plot
def ssp585_plot(data, variable, boundary, period, colormap):
    boundary_plot = boundary.hvplot(
        geo=True,
        color=None,
        alpha=1,
        line_color='black',
        line_width=0.2,
        width=400,
        height=400)
    ssp585_plot = data[variable].hvplot(
        x='lon',
        y='lat',
        rasterize=True,
        geo=True,
        clabel='Percentage Change (%)',
        cmap=colormap,
        width=400,
        height=400,
        frame_width=400,
        frame_height=400
    ).opts(
        title=f'SSP585 ({period})',
        xlabel='Longitude',
        ylabel='Latitude',
        xticks=5,
        yticks=2,
        tools=['hover','wheel_zoom'],
        toolbar='right',
        colorbar_position='bottom',
    )
    return ssp585_plot * boundary_plot

fig8_philippines_ssp585_mid_plot = ssp585_plot(fig8_philippines_ssp585_mid, 'RX1day', fig8_philippines, 'Mid-Century Period (2040-2059)', colormap_sequential)
fig8_manila_ssp585_mid_plot = ssp585_plot(fig8_manila_ssp585_mid, 'RX1day', fig5_manila, 'Mid-Century Period (2040-2059)', colormap_non_sequential)
fig8_cebu_ssp585_mid_plot = ssp585_plot(fig8_cebu_ssp585_mid, 'RX1day', fig4_cebu, 'Mid-Century Period (2040-2059)', colormap_sequential)
fig8_cebu_city_ssp585_mid_plot = ssp585_plot(fig8_cebu_city_ssp585_mid, 'RX1day', fig5_cebu_city, 'Mid-Century Period (2040-2059)', colormap_sequential)
fig8_ncr_ssp585_mid_plot = ssp585_plot(fig8_ncr_ssp585_mid, 'RX1day', fig4_ncr, 'Mid-Century Period (2040-2059)', colormap_non_sequential)

fig8_philippines_ssp585_end_plot = ssp585_plot(fig8_philippines_ssp585_end, 'RX1day', fig8_philippines, 'End-Century Period (2080-2099)', colormap_sequential)
fig8_manila_ssp585_end_plot = ssp585_plot(fig8_manila_ssp585_end, 'RX1day', fig5_manila, 'End-Century Period (2080-2099)', colormap_non_sequential)
fig8_cebu_ssp585_end_plot = ssp585_plot(fig8_cebu_ssp585_end, 'RX1day', fig4_cebu, 'End-Century Period (2080-2099)', colormap_non_sequential)
fig8_cebu_city_ssp585_end_plot = ssp585_plot(fig8_cebu_city_ssp585_end, 'RX1day', fig5_cebu_city, 'End-Century Period (2080-2099)', colormap_sequential)
fig8_ncr_ssp585_end_plot = ssp585_plot(fig8_ncr_ssp585_end, 'RX1day', fig4_ncr, 'End-Century Period (2080-2099)', colormap_non_sequential)

fig8_philippines_cdd_ssp585_mid_plot = ssp585_plot(fig8_cdd_philippines_ssp585_mid, 'CDD', fig8_philippines, 'Mid-Century Period (2040-2059)', cdd_colormap_sequential)
fig8_philippines_cdd_ssp585_end_plot = ssp585_plot(fig8_cdd_philippines_ssp585_end, 'CDD', fig8_philippines, 'End-Century Period (2080-2099)', cdd_colormap_sequential)
fig8_manila_cdd_ssp585_mid_plot = ssp585_plot(fig8_cdd_manila_ssp585_mid, 'CDD', fig5_manila, 'Mid-Century Period (2040-2059)', cdd_colormap_sequential)
fig8_manila_cdd_ssp585_end_plot = ssp585_plot(fig8_cdd_manila_ssp585_end, 'CDD', fig5_manila, 'End-Century Period (2080-2099)', cdd_colormap_sequential)
fig8_ncr_cdd_ssp585_mid_plot = ssp585_plot(fig8_cdd_ncr_ssp585_mid, 'CDD', fig4_ncr, 'Mid-Century Period (2040-2059)', cdd_colormap_sequential)
fig8_ncr_cdd_ssp585_end_plot = ssp585_plot(fig8_cdd_ncr_ssp585_end, 'CDD', fig4_ncr, 'End-Century Period (2080-2099)', cdd_colormap_sequential)
fig8_cebu_cdd_ssp585_mid_plot = ssp585_plot(fig8_cdd_cebu_ssp585_mid, 'CDD', fig4_cebu, 'Mid-Century Period (2040-2059)', cdd_colormap_sequential)
fig8_cebu_cdd_ssp585_end_plot = ssp585_plot(fig8_cdd_cebu_ssp585_end, 'CDD', fig4_cebu, 'End-Century Period (2080-2099)', cdd_colormap_sequential)
fig8_cebu_city_cdd_ssp585_mid_plot = ssp585_plot(fig8_cdd_cebu_city_ssp585_mid, 'CDD', fig5_cebu_city, 'Mid-Century Period (2040-2059)', cdd_colormap_sequential)
fig8_cebu_city_cdd_ssp585_end_plot = ssp585_plot(fig8_cdd_cebu_city_ssp585_end, 'CDD', fig5_cebu_city, 'End-Century Period (2080-2099)', cdd_colormap_sequential)

fig8_philippines_cdd_periods_ssp585_mid_plot = ssp585_plot(fig8_cdd_philippines_ssp585_mid, 'number_of_cdd_periods_with_more_than_5days_per_time_period', fig8_philippines, 'Mid-Century Period (2040-2059)', cdd_colormap_sequential)
fig8_philippines_cdd_periods_ssp585_end_plot = ssp585_plot(fig8_cdd_philippines_ssp585_end, 'number_of_cdd_periods_with_more_than_5days_per_time_period', fig8_philippines, 'End-Century Period (2080-2099)', cdd_colormap_sequential)
fig8_manila_cdd_periods_ssp585_mid_plot = ssp585_plot(fig8_cdd_manila_ssp585_mid, 'number_of_cdd_periods_with_more_than_5days_per_time_period', fig5_manila, 'Mid-Century Period (2040-2059)', cdd_colormap_sequential)
fig8_manila_cdd_periods_ssp585_end_plot = ssp585_plot(fig8_cdd_manila_ssp585_end, 'number_of_cdd_periods_with_more_than_5days_per_time_period', fig5_manila, 'End-Century Period (2080-2099)', cdd_colormap_sequential)
fig8_ncr_cdd_periods_ssp585_mid_plot = ssp585_plot(fig8_cdd_ncr_ssp585_mid, 'number_of_cdd_periods_with_more_than_5days_per_time_period', fig4_ncr, 'Mid-Century Period (2040-2059)', cdd_colormap_sequential)
fig8_ncr_cdd_periods_ssp585_end_plot = ssp585_plot(fig8_cdd_ncr_ssp585_end, 'number_of_cdd_periods_with_more_than_5days_per_time_period', fig4_ncr, 'End-Century Period (2080-2099)', cdd_colormap_sequential)
fig8_cebu_cdd_periods_ssp585_mid_plot = ssp585_plot(fig8_cdd_cebu_ssp585_mid, 'number_of_cdd_periods_with_more_than_5days_per_time_period', fig4_cebu, 'Mid-Century Period (2040-2059)', cdd_colormap_sequential)
fig8_cebu_cdd_periods_ssp585_end_plot = ssp585_plot(fig8_cdd_cebu_ssp585_end, 'number_of_cdd_periods_with_more_than_5days_per_time_period', fig4_cebu, 'End-Century Period (2080-2099)', cdd_colormap_sequential)
fig8_cebu_city_cdd_periods_ssp585_mid_plot = ssp585_plot(fig8_cdd_cebu_city_ssp585_mid, 'number_of_cdd_periods_with_more_than_5days_per_time_period', fig5_cebu_city, 'Mid-Century Period (2040-2059)', cdd_colormap_sequential)
fig8_cebu_city_cdd_periods_ssp585_end_plot = ssp585_plot(fig8_cdd_cebu_city_ssp585_end, 'number_of_cdd_periods_with_more_than_5days_per_time_period', fig5_cebu_city, 'End-Century Period (2080-2099)', cdd_colormap_sequential)

# layout
def fig8_layout(historical_plot, mid, end):
    mid_layout = pn.Row(historical_plot, mid, width=1280)
    end_layout = pn.Row(historical_plot, end, width=1280)
    return mid_layout, end_layout

fig8_philippines_rx1day_mid, fig8_philippines_rx1day_end = fig8_layout(fig8_philippines_historical_plot, fig8_philippines_ssp585_mid_plot, fig8_philippines_ssp585_end_plot)
fig8_philippines_cdd_mid, fig8_philippines_cdd_end = fig8_layout(fig8_philippines_cdd_historical_plot, fig8_philippines_cdd_ssp585_mid_plot, fig8_philippines_cdd_ssp585_end_plot)
fig8_philippines_cdd_periods_mid, fig8_philippines_cdd_periods_end = fig8_layout(fig8_philippines_cdd_period_historical_plot, fig8_philippines_cdd_periods_ssp585_mid_plot, fig8_philippines_cdd_periods_ssp585_end_plot)
fig8_cebu_rx1day_mid, fig8_cebu_rx1day_end = fig8_layout(fig8_cebu_historical_plot, fig8_cebu_ssp585_mid_plot, fig8_cebu_ssp585_end_plot)
fig8_cebu_cdd_mid, fig8_cebu_cdd_end = fig8_layout(fig8_cebu_cdd_historical_plot, fig8_cebu_cdd_ssp585_mid_plot, fig8_cebu_cdd_ssp585_end_plot)
fig8_cebu_cdd_periods_mid, fig8_cebu_cdd_periods_end = fig8_layout(fig8_cebu_cdd_period_historical_plot, fig8_cebu_cdd_periods_ssp585_mid_plot, fig8_cebu_cdd_periods_ssp585_end_plot)
fig8_cebu_city_rx1day_mid, fig8_cebu_city_rx1day_end = fig8_layout(fig8_cebu_city_historical_plot, fig8_cebu_city_ssp585_mid_plot, fig8_cebu_city_ssp585_end_plot)
fig8_cebu_city_cdd_mid, fig8_cebu_city_cdd_end = fig8_layout(fig8_cebu_city_cdd_historical_plot, fig8_cebu_city_cdd_ssp585_mid_plot, fig8_cebu_city_cdd_ssp585_end_plot)
fig8_cebu_city_cdd_periods_mid, fig8_cebu_city_cdd_periods_end = fig8_layout(fig8_cebu_city_cdd_period_historical_plot, fig8_cebu_city_cdd_periods_ssp585_mid_plot, fig8_cebu_city_cdd_periods_ssp585_end_plot)
fig8_ncr_rx1day_mid, fig8_ncr_rx1day_end = fig8_layout(fig8_ncr_historical_plot, fig8_ncr_ssp585_mid_plot, fig8_ncr_ssp585_end_plot)
fig8_ncr_cdd_mid, fig8_ncr_cdd_end = fig8_layout(fig8_ncr_cdd_historical_plot, fig8_ncr_cdd_ssp585_mid_plot, fig8_ncr_cdd_ssp585_end_plot)
fig8_ncr_cdd_periods_mid, fig8_ncr_cdd_periods_end = fig8_layout(fig8_ncr_cdd_period_historical_plot, fig8_ncr_cdd_periods_ssp585_mid_plot, fig8_ncr_cdd_periods_ssp585_end_plot)
fig8_manila_rx1day_mid, fig8_manila_rx1day_end = fig8_layout(fig8_manila_historical_plot, fig8_manila_ssp585_mid_plot, fig8_manila_ssp585_end_plot)
fig8_manila_cdd_mid, fig8_manila_cdd_end = fig8_layout(fig8_manila_cdd_historical_plot, fig8_manila_cdd_ssp585_mid_plot, fig8_manila_cdd_ssp585_end_plot)
fig8_manila_cdd_periods_mid, fig8_manila_cdd_periods_end = fig8_layout(fig8_manila_cdd_period_historical_plot, fig8_manila_cdd_periods_ssp585_mid_plot, fig8_manila_cdd_periods_ssp585_end_plot)

**Dashboard**

In [7]:
# Create sidebar buttons
def create_button_sidebar(name,description):
    button = pn.widgets.Button(
        name=name,
        button_type="default",
        button_style='outline',
        description=description,
        styles={
            'box-shadow':'0px',
            "font-size": "20px",
            "background-color": "#FFF6E9", 
            "border": "2px solid black", 
            "padding": "3px",
            "width": '35%',
            'border-radius':'10px',
        }
    )
    return button

button_introduction = create_button_sidebar('Introduction', ' ')
button1 = create_button_sidebar('Figure 1',
                                'Map of Philippines (left) and Climatogram (right) of the four distinct climate regions of the Philippines.')
button2 = create_button_sidebar('Figure 2',
                                'Number of Landfalling and Non-Landfalling TCs in the Philippines (1951-2013)')
button4 = create_button_sidebar('Figure 4',
                                'Observed Annual Precipitation and Tas (2001-2020)')
button5 = create_button_sidebar('Figure 5',
                                'Observed Rx1Day and TXx in (2001-2020)')
button6 = create_button_sidebar('Figure 8',
                                'Percentage Change in Rx1Day and CDD compared to Historical Period (1995-2014)')
button_static = create_button_sidebar('Static Figures', 'Figures 1, 2, 4, 5, 8')
button_draft = create_button_sidebar('Draft Plots', 'View Draft Plots')

# Create figure 4 and 5 dropdowns
fig4_dropdown = pn.widgets.Select(
    name='Select Region',
    options=['Cebu', 'National Capital Region (NCR)'],
    value='Cebu',
    width=200,
    align='center',
    styles={
        "background-color": "#FFF6E9",
        "border": "2px solid black",
        "border-radius": "10px",
        "padding": "5px",
        "font-size": "15px",
        "font-weight": "bold",
        "color": "black"
    }
)

fig5_dropdown = pn.widgets.Select(
    name='Select Region',
    options=['Cebu', 'Cebu City (Capital)', 'City of Manila', 'National Capital Region (NCR)'],
    value='Cebu',
    width=200,
    align='center',
    styles={
        "background-color": "#FFF6E9",
        "border": "2px solid black",
        "border-radius": "10px",
        "padding": "5px",
        "font-size": "15px",
        "font-weight": "bold",
        "color": "black"
    }
)

# Define hierarchical options for locations, variables, and time periods
options = {
    'Philippines': {
        'Rx1day': ['Mid-Century', 'End-Century'],
        'CDD': ['Mid-Century', 'End-Century'],
        'Number of CDD periods with more than 5 days per time period': ['Mid-Century', 'End-Century'],
    },
    'Cebu': {
        'Rx1day': ['Mid-Century', 'End-Century'],
        'CDD': ['Mid-Century', 'End-Century'],
        'Number of CDD periods with more than 5 days per time period': ['Mid-Century', 'End-Century'],
    },
    'Cebu City': {
        'Rx1day': ['Mid-Century', 'End-Century'],
        'CDD': ['Mid-Century', 'End-Century'],
        'Number of CDD periods with more than 5 days per time period': ['Mid-Century', 'End-Century'],
    },
    'National Capital Region (NCR)': {
        'Rx1day': ['Mid-Century', 'End-Century'],
        'CDD': ['Mid-Century', 'End-Century'],
        'Number of CDD periods with more than 5 days per time period': ['Mid-Century', 'End-Century'],
    },
    'Manila': {
        'Rx1day': ['Mid-Century', 'End-Century'],
        'CDD': ['Mid-Century', 'End-Century'],
        'Number of CDD periods with more than 5 days per time period': ['Mid-Century', 'End-Century'],
    },
}

# Create NestedSelect widget with three levels
fig8_nested_select = pn.widgets.NestedSelect(
    options=options,
    value={'Location': 'Philippines', 'Variable': 'Rx1day', 'Time Period': 'Mid-Century'},
    name='Select Region, Variable, and Time Period',
    levels=['Location', 'Variable', 'Time Period'],
    layout=pn.Row,
    styles={
        "background-color": "#FFF6E9",
        "border": "2px solid black",
        "border-radius": "10px",
        "padding": "5px",
        "font-size": "15px",
        "font-weight": "bold",
        "color": "black"
    }
)

# Create widget to display static figures
static_dropdown = pn.widgets.Select(
    name='Select Figure',
    options=['Figure 1', 'Figure 2', 'Figure 4 (Cebu and NCR)', 
            'Figure 5 (Cebu, Cebu City, Manila, NCR)',
            'Figure 8 (Rx1day)', 'Figure 8 (CDD)', 'Figure 8 (CDD Periods)'],
    value='Figure 1',
    width=200,
    align='center',
    styles={
        "background-color": "#FFF6E9",
        "border": "2px solid black",
        "border-radius": "10px",
        "padding": "5px",
        "font-size": "15px",
        "font-weight": "bold",
        "color": "black"
    }
)

# Create widget to display draft plots
draft_dropdown = pn.widgets.Select(
    name='Select Figure',
    options=['Figure 1', 'Figure 2', 'Figure 4'],
    value='Figure 1',
    width=200,
    align='center',
    styles={
        "background-color": "#FFF6E9",
        "border": "2px solid black",
        "border-radius": "10px",
        "padding": "5px",
        "font-size": "15px",
        "font-weight": "bold",
        "color": "black"
    }
)

In [8]:
fig1_png_path = 'static/fig1_combined.png'
fig2_png_path = 'static/Fig2_Number of TCs.png'
fig4_png_path_cebu = 'static/Fig4_Cebu.png'
fig4_png_path_manila = 'static/Fig4_Manila.png'
fig5_png_path_manila = 'static/Fig5_Manila.png'
fig5_png_path_ncr = 'static/Fig5_NCR.png'
fig5_png_path_cebu_city = 'static/Fig5_Cebu City.png'
fig5_png_path_cebu = 'static/Fig5_Cebu.png'
fig8_png_path_philippines_rx1day_mid = 'static/Fig8_Rx1Day_Philippines_Mid_Century.png'
fig8_png_path_philippines_rx1day_end = 'static/Fig8_Rx1Day_Philippines_End_Century.png'
fig8_png_path_phillippines_cdd_mid = 'static/Fig8_CDD_Philippines_Mid_Century.png'
fig8_png_path_phillippines_cdd_end = 'static/Fig8_CDD_Philippines_End_Century.png'
fig8_png_path_philippines_cdd_period_mid = 'static/Fig8_Periods_Philippines_Mid_Century.png'
fig8_png_path_philippines_cdd_period_end = 'static/Fig8_Periods_Philippines_End_Century.png'
fig8_png_path_manila_rx1day_mid = 'static/Fig8_Rx1Day_Manila_Mid_Century.png'
fig8_png_path_manila_rx1day_end = 'static/Fig8_Rx1Day_Manila_End_Century.png'
fig8_png_path_manila_cdd_mid = 'static/Fig8_CDD_Manila_Mid_Century.png'
fig8_png_path_manila_cdd_end = 'static/Fig8_CDD_Manila_End_Century.png'
fig8_png_path_manila_cdd_period_mid = 'static/Fig8_Periods_Manila_Mid_Century.png'
fig8_png_path_manila_cdd_period_end = 'static/Fig8_Periods_Manila_End_Century.png'
fig8_png_path_ncr_rx1day_mid = 'static/Fig8_Rx1Day_NCR_Mid_Century.png'
fig8_png_path_ncr_rx1day_end = 'static/Fig8_Rx1Day_NCR_End_Century.png'
fig8_png_path_ncr_cdd_mid = 'static/Fig8_CDD_NCR_Mid_Century.png'
fig8_png_path_ncr_cdd_end = 'static/Fig8_CDD_NCR_End_Century.png'
fig8_png_path_ncr_cdd_period_mid = 'static/Fig8_Periods_NCR_Mid_Century.png'
fig8_png_path_ncr_cdd_period_end = 'static/Fig8_Periods_NCR_End_Century.png'
fig8_png_path_cebu_rx1day_mid = 'static/Fig8_Rx1Day_Cebu_Mid_Century.png'
fig8_png_path_cebu_rx1day_end = 'static/Fig8_Rx1Day_Cebu_End_Century.png'
fig8_png_path_cebu_cdd_mid = 'static/Fig8_CDD_Cebu_Mid_Century.png'
fig8_png_path_cebu_cdd_end = 'static/Fig8_CDD_Cebu_End_Century.png'
fig8_png_path_cebu_cdd_period_mid = 'static/Fig8_Periods_Cebu_Mid_Century.png'
fig8_png_path_cebu_cdd_period_end = 'static/Fig8_Periods_Cebu_End_Century.png'
fig8_png_path_cebu_city_rx1day_mid = 'static/Fig8_Rx1Day_Cebu_City_Mid_Century.png'
fig8_png_path_cebu_city_rx1day_end = 'static/Fig8_Rx1Day_Cebu_City_End_Century.png'
fig8_png_path_cebu_city_cdd_mid = 'static/Fig8_CDD_Cebu_City_Mid_Century.png'
fig8_png_path_cebu_city_cdd_end = 'static/Fig8_CDD_Cebu_City_End_Century.png'
fig8_png_path_cebu_city_cdd_period_mid = 'static/Fig8_Periods_Cebu_City_Mid_Century.png'
fig8_png_path_cebu_city_cdd_period_end = 'static/Fig8_Periods_Cebu_City_End_Century.png'

# Descriptive caption images
fig1_caption = 'captions/Fig1_Caption.png'
fig1_info = 'captions/Fig1_info.png'
fig2_caption = 'captions/Fig2_info.png'
fig2_info = 'captions/Fig2_info_2.png'
fig4_mk = 'captions/MKtest.png'
fig4_caption = 'captions/Fig4_info.png'
fig4_info = 'captions/Fig4_info_2.png'
fig4_mk_info = 'captions/Fig4_info_3.png'
fig5_caption = 'captions/Fig5_caption.png'
fig8_caption = 'captions/Fig8_caption.png'
fig8_caption_2 = 'captions/Fig8_caption_2.png'
fig8_caption_3 = 'captions/Fig8_caption_3.png'
fig8_info = 'captions/Fig8_info.png'

def staticimage(image_path, caption_path, info_path):
    return pn.Column(
        pn.Row(
            pn.pane.PNG(caption_path, width=500),
            pn.pane.PNG(info_path, width=500)
        ),
        pn.pane.PNG(image_path, width=1280, height=720))

figure1_static = staticimage(fig1_png_path, fig1_caption, fig1_info)
figure2_static = staticimage(fig2_png_path, fig2_caption, fig2_info)
figure4_static = staticimage(fig4_png_path_cebu, fig4_caption, fig4_info)

def fig8_static(caption, info, ph1, ph2, m1, m2, n1, n2, c1, c2, cc1, cc2):
    return pn.Column(
        pn.Row(
            pn.pane.PNG(caption, width=500),
            pn.pane.PNG(info, width=500)
        ),
        pn.Row(
            pn.pane.PNG(ph1, width=640, height=480),
            pn.pane.PNG(ph2, width=640, height=480)
        ),
        pn.Row(
            pn.pane.PNG(c1, width=640, height=480),
            pn.pane.PNG(c2, width=640, height=480)
        ),
        pn.Row(
            pn.pane.PNG(cc1, width=640, height=480),
            pn.pane.PNG(cc2, width=640, height=480)
        ),
        pn.Row(
            pn.pane.PNG(m1, width=640, height=480),
            pn.pane.PNG(m2, width=640, height=480)
        ),
        pn.Row(
            pn.pane.PNG(n1, width=640, height=480),
            pn.pane.PNG(n2, width=640, height=480)
        )
    )

fig8_rx1day_static = fig8_static(
    fig8_caption, fig8_info,
    fig8_png_path_philippines_rx1day_mid, fig8_png_path_philippines_rx1day_end,
    fig8_png_path_cebu_rx1day_mid, fig8_png_path_cebu_rx1day_end,
    fig8_png_path_manila_rx1day_mid, fig8_png_path_manila_rx1day_end,
    fig8_png_path_ncr_rx1day_mid, fig8_png_path_ncr_rx1day_end,
    fig8_png_path_cebu_city_rx1day_mid, fig8_png_path_cebu_city_rx1day_end
)

fig8_cdd_static = fig8_static(
    fig8_caption_2, fig8_info,
    fig8_png_path_phillippines_cdd_mid, fig8_png_path_phillippines_cdd_end,
    fig8_png_path_cebu_cdd_mid, fig8_png_path_cebu_cdd_end,
    fig8_png_path_cebu_city_cdd_mid, fig8_png_path_cebu_city_cdd_end,
    fig8_png_path_manila_cdd_mid, fig8_png_path_manila_cdd_end,
    fig8_png_path_ncr_cdd_mid, fig8_png_path_ncr_cdd_end,
)

fig8_cdd_period_static = fig8_static(
    fig8_caption_3, fig8_info,
    fig8_png_path_philippines_cdd_period_mid, fig8_png_path_philippines_cdd_period_end,
    fig8_png_path_cebu_cdd_period_mid, fig8_png_path_cebu_cdd_period_end,
    fig8_png_path_cebu_city_cdd_period_mid, fig8_png_path_cebu_city_cdd_period_end,
    fig8_png_path_manila_cdd_period_mid, fig8_png_path_manila_cdd_period_end,
    fig8_png_path_ncr_cdd_period_mid, fig8_png_path_ncr_cdd_period_end,
)

fig1_draft_path = 'drafts/climate_map.png'
fig1_draft_path_climatogram = 'drafts/practice_climatogram.png'
fig1_draft_path_climatogram_2 = 'drafts/climatology_ver2.png'
fig1_draft_caption = 'captions/Fig1_draft_info.png'
fig1_draft_caption_2 = 'captions/Fig1_draft_info_2.png'
fig2_draft_path = 'drafts/TC_frequency.png'
fig2_draft_caption = 'captions/Fig2_draft_info.png'
fig4_draft_path = 'drafts/Fig4_Manila_2.png'
fig4_draft_caption = 'captions/Fig4_draft_info.png'

In [9]:
def CreatePage1():
    # Add image for the introduction
    image_path='drafts/Coverslide.png'
    return pn.pane.PNG(image_path, width=1280, height=720)

def CreatePage2():
    return fig1_layout

def CreatePage3():
    return fig2_plot_with_labels

def CreatePage4():
    # Function to dynamically update the layout based on the dropdown value
    def update_layout(value):
        if value == 'Cebu':
            return fig4_layout_cebu
        elif value == 'National Capital Region (NCR)':
            return fig4_layout_ncr
        else:
            return pn.Row(pn.pane.Markdown("**Please select a valid region.**"))

    # Bind the update_layout function to the dropdown
    dynamic_layout = pn.bind(update_layout, fig4_dropdown)

    # Create the page layout
    return pn.Column(
        fig4_dropdown,
        dynamic_layout,
        align='center'
    )

def CreatePage5():
    return pn.Column(
        fig5_dropdown,
        pn.bind(
            lambda value: 
                fig5_layout_cebu if value == 'Cebu' else
                fig5_layout_cebu_city if value == 'Cebu City (Capital)' else
                fig5_layout_manila if value == 'City of Manila' else
                fig5_layout_ncr, 
            fig5_dropdown
        ),
        align='center'
    )

def CreatePage6():
    return pn.Column(
        fig8_nested_select,
        pn.bind(
            lambda value:
                fig8_philippines_rx1day_mid if value['Location'] == 'Philippines' and value['Variable'] == 'Rx1day' and value['Time Period'] == 'Mid-Century' else
                fig8_philippines_rx1day_end if value['Location'] == 'Philippines' and value['Variable'] == 'Rx1day' and value['Time Period'] == 'End-Century' else
                fig8_cebu_rx1day_mid if value['Location'] == 'Cebu' and value['Variable'] == 'Rx1day' and value['Time Period'] == 'Mid-Century' else
                fig8_cebu_rx1day_end if value['Location'] == 'Cebu' and value['Variable'] == 'Rx1day' and value['Time Period'] == 'End-Century' else
                fig8_cebu_city_rx1day_mid if value['Location'] == 'Cebu City' and value['Variable'] == 'Rx1day' and value['Time Period'] == 'Mid-Century' else
                fig8_cebu_city_rx1day_end if value['Location'] == 'Cebu City' and value['Variable'] == 'Rx1day' and value['Time Period'] == 'End-Century' else
                fig8_manila_rx1day_mid if value['Location'] == 'Manila' and value['Variable'] == 'Rx1day' and value['Time Period'] == 'Mid-Century' else
                fig8_manila_rx1day_end if value['Location'] == 'Manila' and value['Variable'] == 'Rx1day' and value['Time Period'] == 'End-Century' else
                fig8_ncr_rx1day_mid if value['Location'] == 'National Capital Region (NCR)' and value['Variable'] == 'Rx1day' and value['Time Period'] == 'Mid-Century' else
                fig8_ncr_rx1day_end if value['Location'] == 'National Capital Region (NCR)' and value['Variable'] == 'Rx1day' and value['Time Period'] == 'End-Century' else
                fig8_philippines_cdd_mid if value['Location'] == 'Philippines' and value['Variable'] == 'CDD' and value['Time Period'] == 'Mid-Century' else
                fig8_philippines_cdd_end if value['Location'] == 'Philippines' and value['Variable'] == 'CDD' and value['Time Period'] == 'End-Century' else
                fig8_cebu_cdd_mid if value['Location'] == 'Cebu' and value['Variable'] == 'CDD' and value['Time Period'] == 'Mid-Century' else
                fig8_cebu_cdd_end if value['Location'] == 'Cebu' and value['Variable'] == 'CDD' and value['Time Period'] == 'End-Century' else
                fig8_cebu_city_cdd_mid if value['Location'] == 'Cebu City' and value['Variable'] == 'CDD' and value['Time Period'] == 'Mid-Century' else
                fig8_cebu_city_cdd_end if value['Location'] == 'Cebu City' and value['Variable'] == 'CDD' and value['Time Period'] == 'End-Century' else
                fig8_manila_cdd_mid if value['Location'] == 'Manila' and value['Variable'] == 'CDD' and value['Time Period'] == 'Mid-Century' else
                fig8_manila_cdd_end if value['Location'] == 'Manila' and value['Variable'] == 'CDD' and value['Time Period'] == 'End-Century' else
                fig8_ncr_cdd_mid if value['Location'] == 'National Capital Region (NCR)' and value['Variable'] == 'CDD' and value['Time Period'] == 'Mid-Century' else
                fig8_ncr_cdd_end if value['Location'] == 'National Capital Region (NCR)' and value['Variable'] == 'CDD' and value['Time Period'] == 'End-Century' else
                fig8_philippines_cdd_periods_mid if value['Location'] == 'Philippines' and value['Variable'] == 'Number of CDD periods with more than 5 days per time period' and value['Time Period'] == 'Mid-Century' else
                fig8_philippines_cdd_periods_end if value['Location'] == 'Philippines' and value['Variable'] == 'Number of CDD periods with more than 5 days per time period' and value['Time Period'] == 'End-Century' else
                fig8_cebu_cdd_periods_mid if value['Location'] == 'Cebu' and value['Variable'] == 'Number of CDD periods with more than 5 days per time period' and value['Time Period'] == 'Mid-Century' else
                fig8_cebu_cdd_periods_end if value['Location'] == 'Cebu' and value['Variable'] == 'Number of CDD periods with more than 5 days per time period' and value['Time Period'] == 'End-Century' else
                fig8_cebu_city_cdd_periods_mid if value['Location'] == 'Cebu City' and value['Variable'] == 'Number of CDD periods with more than 5 days per time period' and value['Time Period'] == 'Mid-Century' else
                fig8_cebu_city_cdd_periods_end if value['Location'] == 'Cebu City' and value['Variable'] == 'Number of CDD periods with more than 5 days per time period' and value['Time Period'] == 'End-Century' else
                fig8_manila_cdd_periods_mid if value['Location'] == 'Manila' and value['Variable'] == 'Number of CDD periods with more than 5 days per time period' and value['Time Period'] == 'Mid-Century' else
                fig8_manila_cdd_periods_end if value['Location'] == 'Manila' and value['Variable'] == 'Number of CDD periods with more than 5 days per time period' and value['Time Period'] == 'End-Century' else
                fig8_ncr_cdd_periods_mid if value['Location'] == 'National Capital Region (NCR)' and value['Variable'] == 'Number of CDD periods with more than 5 days per time period' and value['Time Period'] == 'Mid-Century' else
                fig8_ncr_cdd_periods_end if value['Location'] == 'National Capital Region (NCR)' and value['Variable'] == 'Number of CDD periods with more than 5 days per time period' and value['Time Period'] == 'End-Century' else
                pn.Row(pn.pane.Markdown("**Please select a valid region, variable, and time period.**")),
            fig8_nested_select
        ),
        align='center'
    )

def CreatePage7():
    return pn.Column(static_dropdown,
        pn.bind(
            lambda value:
                figure1_static if value == 'Figure 1' else
                figure2_static if value == 'Figure 2' else
                pn.Column(
                    figure4_static,
                    pn.pane.PNG(fig4_png_path_manila, width=1280, height=720),
                    pn.pane.PNG(fig4_mk_info),
                    pn.pane.PNG(fig4_mk, width=1080)
                ) if value == 'Figure 4 (Cebu and NCR)' else
                pn.Column(
                    pn.Row(
                        pn.pane.PNG(fig5_caption, width=500),
                        pn.pane.PNG(fig4_info, width=500)
                    ),
                    pn.Row(
                        pn.pane.PNG(fig5_png_path_cebu, width=640, height=480),
                        pn.pane.PNG(fig5_png_path_cebu_city, width=640, height=480)
                    ),
                    pn.Row(
                        pn.pane.PNG(fig5_png_path_manila, width=640, height=480),
                        pn.pane.PNG(fig5_png_path_ncr, width=640, height=480)
                    ),
                ) if value == 'Figure 5 (Cebu, Cebu City, Manila, NCR)' else
                fig8_rx1day_static if value == 'Figure 8 (Rx1day)' else
                fig8_cdd_static if value == 'Figure 8 (CDD)' else
                fig8_cdd_period_static if value == 'Figure 8 (CDD Periods)' else
                None,
            static_dropdown),
        align='center'
    )

def CreatePage8():
    return pn.Column(draft_dropdown,
        pn.bind(
            lambda value:
                pn.Row(
                    pn.Column(
                        pn.pane.PNG(fig1_draft_caption, align='center'),
                        pn.Row(
                            pn.pane.PNG(fig1_draft_path, height=720, width=400),
                            pn.pane.PNG(fig1_draft_path_climatogram,height=720, width=400)
                        )),
                    pn.Column(
                        pn.pane.PNG(fig1_draft_caption_2, align='center'), 
                        pn.pane.PNG(fig1_draft_path_climatogram_2,height=720, width=640)
                    ),
                    ) if value == 'Figure 1' else
                pn.Column(
                    pn.pane.PNG(fig2_draft_caption, align='center'),
                    pn.pane.PNG(fig2_draft_path, height=720)
                ) if value == 'Figure 2' else
                pn.Column(
                    pn.pane.PNG(fig4_draft_caption, align='center'),
                    pn.pane.PNG(fig4_draft_path, width=1280, height=720)
                ) if value == 'Figure 4' else
                None,
            draft_dropdown),
        align='center'
    )

In [10]:
mapping = {
    'Introduction': CreatePage1(),
    'Figure1': CreatePage2(),
    'Figure2': CreatePage3(),
    'Figure4': CreatePage4(),
    'Figure5': CreatePage5(),
    'Figure8': CreatePage6(),
    'StaticImages': CreatePage7(),
    'DraftImages': CreatePage8()
}

# Create sidebar layout
sidebar = pn.Column(
    pn.pane.Markdown('## Navigation', width=200),
    button_introduction,
    button1,
    button2,
    button4,
    button5,
    button6,
    button_static,
    button_draft,
    width=500,
    align='center'
)

# Create main area layout
main_area = pn.Column(
    mapping["Introduction"],
    width=800,
)

def show_page(page_key):
    main_area.clear()
    main_area.append(mapping[page_key])

button_introduction.on_click(lambda event: show_page("Introduction"))
button1.on_click(lambda event: show_page("Figure1"))
button2.on_click(lambda event: show_page("Figure2"))
button4.on_click(lambda event: show_page("Figure4"))
button5.on_click(lambda event: show_page("Figure5"))
button6.on_click(lambda event: show_page("Figure8"))
button_static.on_click(lambda event: show_page("StaticImages"))
button_draft.on_click(lambda event: show_page("DraftImages"))

Watcher(inst=Button(button_style='outline', description='View Draft Plots', name='Draft Plots', sizing_mode='stretch_width', styles={'box-shadow': '0px', ...}), cls=<class 'panel.widgets.button.Button'>, fn=<function <lambda> at 0x157984280>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [11]:
template = pn.template.BootstrapTemplate(
    title=" Eileen's Internship Dashboard",
    sidebar=[sidebar],
    main=[main_area],
    header_background="#B35C3E", 
    sidebar_width=250,
)

pn.serve(template)

Launching server at http://localhost:54828
